In [ ]:
import csv
import numpy as np

In [ ]:
def calc_score_diff(play):
    """Calculate the score differential of the team with possession.

    Parameters:
    play(Dict): The play object.

    Returns:
    int: The score differential of the team with possession.
    """

    return int(play['posteam_score']) - int(play['defteam_score'])

def get_yrdln_int(play):
    """Given a play, get the line of scrimmage as an integer.

    Parameters:
    play(Dict): The play object.

    Returns:
    int: The yard line as an integer.
    """
    
    return int(play['yrdln'].split(' ')[-1])

def calc_seconds_since_halftime(play):
    """Calculate the number of seconds elapsed since halftime.

    Parameters:
    play(Dict): The play object.

    Returns:
    int: The number of seconds elapsed since halftime of that play.
    """
    
    if play['game_seconds_remaining'] == '1e3':
        return 1800 - 1000
    return max(0, 1800-int(play['game_seconds_remaining']))

def calc_field_pos_score(play):
    """Calculate the field position score for a play.

    Parameters:
    play(Dict): The play object.

    Returns:
    float: The "field position score" for a given play,
           used to calculate the surrender index.
    """

    try:
        if '50' in play['yrdln']:
            return (1.1) ** 10.
        if play['posteam'] in play['yrdln']:
            return max(1., (1.1)**(get_yrdln_int(play) - 40))
        else:
            return (1.2)**(50 - get_yrdln_int(play)) * ((1.1)**(10))
    except BaseException:
        return 0.


def calc_yds_to_go_multiplier(play):
    """Calculate the yards to go multiplier for a play.

    Parameters:
    play(Dict): The play object.

    Returns:
    float: The "yards to go multiplier" for a given play,
           used to calculate the surrender index.
    """

    if int(play['ydstogo']) >= 10:
        return 0.2
    elif int(play['ydstogo']) >= 7:
        return 0.4
    elif int(play['ydstogo']) >= 4:
        return 0.6
    elif int(play['ydstogo']) >= 2:
        return 0.8
    else:
        return 1.


def calc_score_multiplier(play):
    """Calculate the score multiplier for a play.

    Parameters:
    play(Dict): The play object.

    Returns:
    float: The "score multiplier" for a given play,
           used to calculate the surrender index.
    """

    score_diff = calc_score_diff(play)

    if score_diff > 0:
        return 1.
    elif score_diff == 0:
        return 2.
    elif score_diff < -8.:
        return 3.
    else:
        return 4.


def calc_clock_multiplier(play):
    """Calculate the clock multiplier for a play.

    Parameters:
    play(Dict): The play object.

    Returns:
    float: The "clock multiplier" for a given play,
           used to calculate the surrender index.
    """

    if calc_score_diff(play) <= 0 and int(play['qtr']) > 2:
        seconds_since_halftime = calc_seconds_since_halftime(play)
        return ((seconds_since_halftime * 0.001) ** 3.) + 1.
    else:
        return 1.


def calc_surrender_index(play):
    """Calculate the surrender index for a play.

    Parameters:
    play(Dict): The play object.

    Returns:
    float: The surrender index for a given play.
    """

    return calc_field_pos_score(play) * calc_yds_to_go_multiplier(
        play) * calc_score_multiplier(play) * calc_clock_multiplier(play)

In [ ]:
surrender_indices = []

for year in range(2009, 2019):
    with open('nflscrapR-data/play_by_play_data/regular_season/reg_pbp_' + str(year) + '.csv', 'r') as f:
        data = csv.DictReader(f)
        for play in data:
            if play['play_type'] == 'punt':
                surrender_indices.append(calc_surrender_index(play))

    with open('nflscrapR-data/play_by_play_data/post_season/post_pbp_' + str(year) + '.csv', 'r') as f:
        data = csv.DictReader(f)
        for play in data:
            if play['play_type'] == 'punt':
                surrender_indices.append(calc_surrender_index(play))

In [ ]:
len(surrender_indices)

In [ ]:
np_surrender_indices = np.array(surrender_indices)

In [ ]:
np.save('2009-2018_surrender_indices.npy', np_surrender_indices)